In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Parse proteome data

From raw data files. 

Need to run the following first:

`wget http://rest.kegg.jp/link/rn/ko -O "./data/KEGG/ko-to-rn-api.txt"`

`wget http://rest.kegg.jp/link/ec/ko -O "./data/KEGG/ko-to-ec-api.txt"`

`wget http://rest.kegg.jp/link/rn/ec -O "./data/KEGG/ec-to-ko-api.txt"`

## Load modules

In [2]:
import re
import pandas as pd
import numpy as np
import matplotlib as plot
import seaborn as sns
import os
import copy

In [3]:
from itertools import cycle
from collections import defaultdict

## File paths

In [4]:
!pwd -P

/home/jovyan/work/make_graph


In [5]:
# In
percolator_folder    = '/home/jovyan/data/drive/UpdateSearches/CruxOutput/'
sample_metadata_file = '/home/jovyan/data/drive/UpdatedMetaData.txt'

cluster_file            = "/home/jovyan/data/drive/Annotations/AccessionsClusteredAt100ID.tab"
taxa_file               = "/home/jovyan/data/drive/Annotations/scaffold2bin.tsv"

bact_eggnogg_file_path  = "/home/jovyan/data/drive/Annotations/DiamondPreterm.emapper.annotations"
human_eggnogg_file_path = "/home/jovyan/data/drive/Annotations/uniprot-proteome_UP000005640_Cluster100.emapper.annotations"

In [6]:
# KEGG translators
ko_rn = "/home/jovyan/data/KEGG/ko-to-rn-api.txt"
ko_ec = "/home/jovyan/data/KEGG/ko-to-ec-api.txt"
ec_rn = "/home/jovyan/data/KEGG/ec-to-ko-api.txt"

In [7]:
# Out
baby_entities_file = '/home/jovyan/data/import/baby_entities.tsv'
sample_entities_file = '/home/jovyan/data/import/sample_entities.tsv'
protein_entities_file = '/home/jovyan/data/import/protein_entities.tsv'

replicates_protein_sample_file_name = '/home/jovyan/data/import/protein_replicate_sample_relationship.tsv'
aggregated_protein_sample_file_name = '/home/jovyan/data/import/protein_aggregated_sample_relationship.tsv'

baby_sample_file_name = '/home/jovyan/data/import/baby_sample_relationship.tsv'
protein_reaction_file_name = '/home/jovyan/data/import/protein_reaction_relationship.tsv'

In [8]:
def list_to_string(x):
    return ",".join([str(i) for i in x])

## Babies

In [9]:
def get_first(x):
    return x.values[0]
    
def make_list(x):
    r = []
    for l in x.values:
        if not pd.isnull(l):
            r += ["-".join([x.strip() for x in l.split(",")])]
    
    return "//".join(r)

baby_data = pd.read_csv(sample_metadata_file, sep='\t', usecols=['Baby',
       'Nec', 'NecDiagnosis (DOL)', 'BirthAge', 'Feeding', 'Delivery', 'birth weight  (g)', 'Infection',
       'InfectionDiagnosis (DOL)', 'sex', 'AntibioticTreament'], na_values=["No Info", 'n/a'])
baby_data.drop_duplicates(inplace=True)
    
baby_data = baby_data.groupby("Baby").aggregate({'Nec':get_first, 
                                             'NecDiagnosis (DOL)':get_first, 
                                             'BirthAge':get_first, 
                                             'Feeding':get_first, 
                                             'Delivery':get_first, 
                                             'birth weight  (g)':get_first, 
                                             'Infection':get_first,
                                             'InfectionDiagnosis (DOL)':make_list, 
                                             'sex':get_first, 
                                             'AntibioticTreament':make_list})

In [10]:
baby_data.index.name = "ID"
baby_data.columns = ['NEC', 
                     'NEC_DIAGNOSIS_DOL', 
                     'BIRTH_AGE',
                     'FEEDING',
                     'DELIVERY',
                     'BIRTH_WEIGHT',
                     'INFECTION',
                     'INFECTION_DIAGNOSIS_DOL',
                     'SEX',
                     'ANTIBIOTIC_TREATMENT']

In [11]:
baby_data.to_csv(baby_entities_file, encoding="utf-8", quoting=3, sep='\t', index=True)

## Samples

In [12]:
meta_data = pd.read_csv(sample_metadata_file, sep='\t', 
                        usecols=["ProteomeRun1", "ProteomeRun2", "Day", 
                                 "GestationalAgeInWeeks", "Baby"])
meta_data.dropna(subset=['ProteomeRun1', 'ProteomeRun2'], inplace=True)

a = meta_data.set_index('ProteomeRun1')
del a['ProteomeRun2']

b = meta_data.set_index('ProteomeRun2')
del b['ProteomeRun1']

meta_data = pd.concat([a, b])
meta_data.sort_index(inplace=True)

In [13]:
def get_sample_id(x):
    pattern = re.compile("^(?P<number>[0-9]*)\_(?P<baby>Baby[0-9]*)\_(?P<day>Day[0-9]*)\_(?:(?P<sample>[0-9]*)\_)*50ug_DF_Orbi_Elite_Full_30K_$")
    g = re.match(pattern=pattern, string=x).groupdict()
    measurment = g["number"] + "_" + g["baby"] + "_" + g["day"] 
    if not g["sample"] is None:
        measurment += "_" + g["sample"]
    return measurment

def get_file_id(x):
    pattern = re.compile("^(?P<number>[0-9]*)\_(?P<baby>Baby[0-9]*)\_(?P<day>Day[0-9]*)\_(?:(?P<sample>[0-9]*)\_)*50ug_DF_Orbi_Elite_Full_30K_$")
    g = re.match(pattern=pattern, string=x).groupdict()
    measurment = g["number"] + "_" + g["baby"] + "_" + g["day"] 
    return measurment


def get_day_sample(x):
    pattern = re.compile("^(?P<number>[0-9]*)\_(?P<baby>Baby[0-9]*)\_(?P<day>Day[0-9]*)\_(?:(?P<sample>[0-9]*)\_)*50ug_DF_Orbi_Elite_Full_30K_$")
    g = re.match(pattern=pattern, string=x).groupdict()
    if not g["sample"] is None:
        return g["sample"]
    else:
        return 1

meta_data["ID"] = meta_data.index.to_series().apply(get_sample_id)
meta_data["Day-Sample"] = meta_data.index.to_series().apply(get_day_sample)
meta_data["FileID"] = meta_data.index.to_series().apply(get_file_id)

In [14]:
new_ids = []
for s, row in meta_data.iterrows():
    new_ids.append("S_" + ".".join([str(row[x]) for x in ["Baby", "Day", "Day-Sample"]]))

meta_data.index = new_ids

In [15]:
meta_data.head()

,Day,Baby,GestationalAgeInWeeks,ID,Day-Sample,FileID
S_3.11.1,11,3,27,20140509_Baby3_Day11,1,20140509_Baby3_Day11
S_3.11.1,11,3,27,20140510_Baby3_Day11,1,20140510_Baby3_Day11
S_3.15.1,15,3,28,20140512_Baby3_Day15_1,1,20140512_Baby3_Day15
S_3.21.1,21,3,29,20140514_Baby3_Day21_1,1,20140514_Baby3_Day21
S_3.21.1,21,3,29,20140515_Baby3_Day21_1,1,20140515_Baby3_Day21


In [16]:
len(set(meta_data["ID"])) # 182

182

In [17]:
len(set(meta_data.index)) # 91

91

In [18]:
sample_data = meta_data.copy()
sample_data["rep"] = 1
sample_data = sample_data.groupby(level=0).aggregate({"Day":get_first, 
                                                    "GestationalAgeInWeeks":get_first, 
                                                    "Baby":get_first, 
                                                    "rep":sum})

In [19]:
sample_data.index.name = "ID"
sample_data.columns = ["DAY", "GESTATION_WEEK", "BABY", "NUM_REPS"]

In [20]:
sample_data.to_csv(sample_entities_file, encoding="utf-8", quoting=3, sep='\t', index=True)

In [21]:
sample_data.head()

,DAY,GESTATION_WEEK,BABY,NUM_REPS
ID,,,,
S_19.12.1,12,25,19,2
S_19.16.1,16,26,19,2
S_19.20.1,20,26,19,2
S_19.26.1,26,27,19,2
S_19.31.1,31,28,19,2


In [22]:
# key for file IDs to new IDs
ids = pd.DataFrame(meta_data["FileID"]).reset_index().set_index("FileID").squeeze().to_dict()

In [23]:
len(ids.keys()) # 182

182

In [24]:
len(set(ids.values())) # 91

91

## Baby-Sample

In [25]:
sample_data[["BABY"]].to_csv(baby_sample_file_name, encoding="utf-8", quoting=3, sep='\t', index=True)

## Measurements

In [26]:
#Import all Percolator files and place them in dictionary
FilesDict = defaultdict(dict)
for (dirpath, dirnames, filenames) in os.walk(percolator_folder):
    for name in filenames:
        tmp = name.split(".")
        sample_name = tmp[0]

        if name.endswith("percolator.log"):
            FilesDict[sample_name]['log'] = name
        elif name.endswith("percolator.target.proteins.txt"):
            FilesDict[sample_name]['protein'] = name
        elif name.endswith("percolator.target.psms"):
            FilesDict[sample_name]['psm'] = name
        elif name.endswith(".02.spectral-counts.target.txt"):
            FilesDict[sample_name]['NSAF'] = name
        else:
            print(name, "I don't know why I'm here")

In [27]:
len(FilesDict) # 182

182

### Sample-Protein links

In [28]:
UniqueCutoff = 1
with open(replicates_protein_sample_file_name, "w") as out:
    out.write("SAMPLE\tPROTEIN\tQ_VALUE\tSPEC_COUNT_UNIQUE\tNSAF\n")
    for k in FilesDict:
        
        sample = ids[k]
        
        protein_file = os.path.join(percolator_folder, FilesDict[k]["protein"])
        protein_df = pd.read_csv(protein_file, sep="\t", 
                                 usecols=["ProteinId", "q-value", "spec_count_unique"], 
                                 index_col=0)
        # filter by species count
        protein_df = protein_df[protein_df["spec_count_unique"] >= UniqueCutoff]

        nsaf_file = os.path.join(percolator_folder, FilesDict[k]["NSAF"])
        nsaf_df = pd.read_csv(nsaf_file, sep="\t", 
                              usecols=["protein id", "NSAF"], 
                              index_col=0)
        df = protein_df.join(nsaf_df)
        df.index = df.index.to_series().apply(lambda x: '"%s"'%x)
        df.index = pd.MultiIndex.from_tuples(list(zip(cycle([sample]), df.index.values)))
        
        df.to_csv(out, header=False, sep="\t", quoting=3)


In [29]:
df = pd.read_csv(replicates_protein_sample_file_name, sep="\t")
df.sort_values(["SAMPLE", "PROTEIN"], inplace=True)

In [30]:
df.isnull().sum()

SAMPLE               0
PROTEIN              0
Q_VALUE              0
SPEC_COUNT_UNIQUE    0
NSAF                 0
dtype: int64

In [31]:
df.duplicated().sum() # 0

0

In [32]:
df.head()

,SAMPLE,PROTEIN,Q_VALUE,SPEC_COUNT_UNIQUE,NSAF
1177220,S_19.12.1,b019-d010_scaffold_0_100,0.020592,1,0.000009
1172887,S_19.12.1,b019-d010_scaffold_0_104,0.015775,4,0.000009
2208266,S_19.12.1,b019-d010_scaffold_0_104,0.021652,1,0.000005
1185629,S_19.12.1,b019-d010_scaffold_0_105,0.027842,1,0.000004
1178910,S_19.12.1,b019-d010_scaffold_0_106,0.021569,1,0.000002


In [33]:
df[df["SAMPLE"]=="S_93.8.2"]

,SAMPLE,PROTEIN,Q_VALUE,SPEC_COUNT_UNIQUE,NSAF


In [34]:
df[["SAMPLE", "PROTEIN"]].duplicated().sum() # 903424

872679

In [35]:
# Merge technical replicates into one protein per sample. 
# NSAF --> mean
# q-value --> min
agg = df.groupby(["SAMPLE", "PROTEIN"]).aggregate({"Q_VALUE":min, "NSAF":pd.np.mean})

In [36]:
agg.head()

Q_VALUE      NSAF
SAMPLE    PROTEIN                                     
S_19.12.1 b019-d010_scaffold_0_100  0.020592  0.000009
          b019-d010_scaffold_0_104  0.015775  0.000007
          b019-d010_scaffold_0_105  0.027842  0.000004
          b019-d010_scaffold_0_106  0.021569  0.000002
          b019-d010_scaffold_0_108  0.021922  0.000009

In [37]:
agg.to_csv(aggregated_protein_sample_file_name, sep="\t", quoting=3)

In [38]:
aggregated_protein_sample_file_name

'/home/jovyan/data/import/protein_aggregated_sample_relationship.tsv'

## Proteins

In [39]:
proteins = list(set(agg.index.levels[1]))

In [40]:
len(proteins) # 652312

652312

### Protein clusters

In [41]:
clusters = pd.read_csv(cluster_file, sep="\t", 
                       header=None, usecols=[0, 2], 
                       index_col=[1], names=['cluster', 'protein'])

In [42]:
clusters = clusters.loc[proteins]

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


In [43]:
def scaffold_id(x):
    x = x.strip()
    x = x[:x.rfind("_")]
    return x

clusters["scaffold"] = clusters.index.to_series().apply(scaffold_id)

In [44]:
clusters.head()

,cluster,scaffold
protein,,
b003-d078_scaffold_1413_2,Cluster516100,b003-d078_scaffold_1413
70_007_scaffold_158_24,Cluster51958,70_007_scaffold_158
b021-d012_scaffold_9_87,Cluster471375,b021-d012_scaffold_9
b003-d026_scaffold_583_1,Cluster194118,b003-d026_scaffold_583
60_005_scaffold_475_1,Cluster246519,60_005_scaffold_475


In [45]:
clusters.shape[0] == len(proteins)

True

### Taxa

In [46]:
taxa_df = pd.read_csv(taxa_file, sep="\t", usecols=[0, 3], index_col=0, names=["scaffold", "species"], header=0)
taxa_df['genus'] = taxa_df["species"].str.split(' ', 1).str[0]

In [47]:
taxa_df.head()

,species,genus
scaffold,,
b003-d013_scaffold_203,Staphylococcus epidermidis 2,Staphylococcus
b003-d013_scaffold_211,Staphylococcus epidermidis 2,Staphylococcus
b003-d013_scaffold_52,Staphylococcus epidermidis 2,Staphylococcus
b003-d013_scaffold_40,Staphylococcus epidermidis 2,Staphylococcus
b003-d013_scaffold_316,Staphylococcus epidermidis 2,Staphylococcus


In [48]:
proteins_dat = clusters.join(taxa_df, on="scaffold")

In [49]:
proteins_dat.shape # 652,312

(652312, 4)

In [50]:
proteins_dat.head()

,cluster,scaffold,species,genus
protein,,,,
b003-d078_scaffold_1413_2,Cluster516100,b003-d078_scaffold_1413,NaN,NaN
70_007_scaffold_158_24,Cluster51958,70_007_scaffold_158,Enterococcus faecalis 2,Enterococcus
b021-d012_scaffold_9_87,Cluster471375,b021-d012_scaffold_9,Propionibacterium acnes,Propionibacterium
b003-d026_scaffold_583_1,Cluster194118,b003-d026_scaffold_583,Klebsiella pneumoniae,Klebsiella
60_005_scaffold_475_1,Cluster246519,60_005_scaffold_475,Klebsiella sp,Klebsiella


In [51]:
version = "KO.bacteria"
bact_eggnog_df = pd.read_csv(bact_eggnogg_file_path, sep="\t", header=None, comment="#",
                        names=["protein", version], usecols=[0, 6], index_col=0)
bact_eggnog_df.dropna(axis=0, how='any', inplace=True)
bact_eggnog_df[version] = bact_eggnog_df[version].str.split(",")

In [52]:
version = "KO.human"
human_eggnog_df = pd.read_csv(human_eggnogg_file_path, sep="\t", header=None, comment="#",
                        names=["protein", version], usecols=[0, 6], index_col=0)
human_eggnog_df.dropna(axis=0, how='any', inplace=True)
human_eggnog_df[version] = human_eggnog_df[version].str.split(",")

In [53]:
proteins_dat = proteins_dat.join(bact_eggnog_df).join(human_eggnog_df)

In [54]:
def combine_cols(row, col1, col2):
    result = []
    for l in (row[col1], row[col2]):
        if type(l) == list:
            result += l
    return list(set(result))


proteins_dat["KO"] = proteins_dat.apply(combine_cols, col1="KO.bacteria", col2="KO.human", axis=1)

In [55]:
del proteins_dat["KO.bacteria"]
del proteins_dat["KO.human"]
del proteins_dat["scaffold"]

In [56]:
protein_entities = proteins_dat.copy() # use for saving protein entities

In [57]:
human_overlap = list(set(human_eggnog_df.index).intersection(proteins_dat.index))
protein_entities.loc[human_overlap, "genus"] = "Homo"
protein_entities.loc[human_overlap, "species"] = "Homo sapiens"

protein_entities.loc[human_overlap, "Taxa"] = "human"
protein_entities.loc[~protein_entities.index.isin(human_overlap), "Taxa"] = "bacteria"

In [58]:
protein_entities["KO"] = protein_entities["KO"].apply(list_to_string)

In [59]:
protein_entities.head()

,cluster,species,genus,KO,Taxa
protein,,,,,
b003-d078_scaffold_1413_2,Cluster516100,NaN,NaN,K18568,bacteria
70_007_scaffold_158_24,Cluster51958,Enterococcus faecalis 2,Enterococcus,"K02470,K02622",bacteria
b021-d012_scaffold_9_87,Cluster471375,Propionibacterium acnes,Propionibacterium,,bacteria
b003-d026_scaffold_583_1,Cluster194118,Klebsiella pneumoniae,Klebsiella,K03528,bacteria
60_005_scaffold_475_1,Cluster246519,Klebsiella sp,Klebsiella,"K00863,K05878",bacteria


In [60]:
protein_entities.index.name = "ID"
protein_entities.columns = ['CLUSTER',
                            'SPECIES',
                            'GENUS', 
                            'KO', 
                            'TAXA']


In [61]:
protein_entities['SPECIES'] = protein_entities['SPECIES'].apply(lambda x: '"%s"'%x)

In [62]:
protein_entities.to_csv(protein_entities_file, encoding="utf-8", quoting=3, sep='\t', index=True)

In [63]:
del protein_entities

In [64]:
proteins_dat = proteins_dat[["KO"]]

In [65]:
proteins_dat.head()

,KO
protein,
b003-d078_scaffold_1413_2,[K18568]
70_007_scaffold_158_24,"[K02470, K02622]"
b021-d012_scaffold_9_87,[]
b003-d026_scaffold_583_1,[K03528]
60_005_scaffold_475_1,"[K00863, K05878]"


In [66]:
# delete no KO -term
ids = proteins_dat.apply(lambda x:x["KO"]==[], axis=1)
proteins_dat = proteins_dat[~ids]

In [67]:
KOs = pd.DataFrame(proteins_dat['KO'].apply(pd.Series, 1).stack())
KOs.index = KOs.index.droplevel(-1)
KOs.columns = ['KO']

In [68]:
KOs.head()

,KO
protein,
b003-d078_scaffold_1413_2,K18568
70_007_scaffold_158_24,K02470
70_007_scaffold_158_24,K02622
b003-d026_scaffold_583_1,K03528
60_005_scaffold_475_1,K00863


In [69]:
# KEGG translators
f = lambda x:x.split(":")[1]
df_ko_rn = pd.read_csv(ko_rn, sep="\t", header=None, index_col=0, names=["ko", "rn-ko"], converters={0:f, 1:f})
df_ko_ec = pd.read_csv(ko_ec, sep="\t", header=None, index_col=0, names=["ko", "ec"], converters={0:f, 1:f})
df_ec_rn = pd.read_csv(ec_rn, sep="\t", header=None, index_col=0, names=["ec", "rn-ec"], converters={0:f, 1:f})

In [70]:
KOs = KOs.join(df_ko_rn, on="KO")
KOs = KOs.join(df_ko_ec, on="KO")
KOs = KOs.join(df_ec_rn, on="ec")

In [71]:
def combine_cols_v2(row, col1, col2):
    result = []
    for l in (row[col1], row[col2]):
        if not pd.isnull(l):
            result += [l]
    if len(result) > 0:
        return list(set(result))
    else:
        return pd.np.NAN

In [72]:
KOs["REACTION"] = KOs.apply(combine_cols_v2, col1="rn-ko", col2="rn-ec", axis=1)

In [73]:
KOs.head()

,KO,rn-ko,ec,rn-ec,REACTION
protein,,,,,
b003-d078_scaffold_1413_2,K18568,R10747,NaN,NaN,[R10747]
70_007_scaffold_158_24,K02470,NaN,5.6.2.2,NaN,NaN
70_007_scaffold_158_24,K02622,NaN,5.6.2.2,NaN,NaN
b003-d026_scaffold_583_1,K03528,NaN,NaN,NaN,NaN
60_005_scaffold_475_1,K00863,R01011,2.7.1.28,R01059,"[R01059, R01011]"


In [74]:
# number missing REACTION should be smaller than rn and rn-ec
KOs.isnull().sum()

KO               0
rn-ko       392833
ec          299101
rn-ec       340802
REACTION    313540
dtype: int64

In [75]:
KOs = KOs[["KO", "REACTION"]]

In [76]:
KOs.set_index(['KO'], append=True, inplace=True)

In [77]:
KOs.dropna(subset=["REACTION"], inplace=True)

In [78]:
KOs.shape # 3,331,712

(3331712, 1)

In [79]:
KOs.head()

REACTION
protein                   KO                      
b003-d078_scaffold_1413_2 K18568          [R10747]
60_005_scaffold_475_1     K00863  [R01059, R01011]
                          K00863          [R01011]
                          K00863  [R01011, R07636]
                          K00863          [R01059]

In [80]:
edges = KOs.groupby(level=[0, 1]).aggregate(lambda x:set(x.sum()))

In [81]:
edges.shape # 258719

(258719, 1)

In [82]:
edges.head()

REACTION
protein               KO                                      
31_003_scaffold_0_1   K12972  {R00465, R01392, R02527, R01388}
31_003_scaffold_0_103 K01424                  {R00485, R06134}
                      K05597  {R00256, R06134, R01579, R00485}
31_003_scaffold_0_104 K02517                  {R12193, R05146}
                      K12974                          {R10906}

In [83]:
# faster to write as is than to use 
# edges = pd.DataFrame(KOs['REACTION'].apply(pd.Series, 1).stack())
with open(protein_reaction_file_name, "w") as handle:
    handle.write("PROTEIN\tREACTION\tKO\n")
    for (protein, KO), row in edges.iterrows():
        for reaction in row["REACTION"]:
            handle.write("%s\t%s\t%s\n"%(protein, reaction, KO))